In [1]:
%matplotlib inline
import matplotlib.pyplot as plot
from IPython import display
from __future__ import print_function

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, Embedding, BatchNormalization
from keras.optimizers import RMSprop, Adam
from keras.regularizers import l2
from keras.utils.data_utils import get_file
import numpy as np
import numpy.random as nr
import random
import glob
import sys
import os
import re

sys.path.append('../src')
from my_keras_utilities import (get_available_gpus, 
                                load_model_and_history, 
                                save_model_and_history, 
                                TrainingPlotter)

os.makedirs('../../models',exist_ok=True)
np.set_printoptions(precision=3, linewidth=120)

Using TensorFlow backend.


In [2]:
import keras.backend as K

# K.set_image_data_format('channels_first')
K.set_floatx('float32')

print('Backend:        {}'.format(K.backend()))
print('Data format:    {}'.format(K.image_data_format()))
print('Available GPUS:', get_available_gpus())

Backend:        tensorflow
Data format:    channels_last
Available GPUS: ['/gpu:0']


http://philipperemy.github.io/keras-stateful-lstm/

In [3]:
n_samples = 2000
seq_size = 50 

Xtra = np.zeros((n_samples, seq_size, 1), np.float32)
ytra = np.zeros((n_samples, 1), np.float32)
tii = nr.choice(np.arange(n_samples), replace=False, size=n_samples//2)
Xtra[tii, 0] = 1
ytra[tii] = 1

print(tii.shape, Xtra[:,0].sum(0), Xtra.shape, ytra.shape)

(1000,) [ 1000.] (2000, 50, 1) (2000, 1)


In [4]:
from sklearn.model_selection import train_test_split

X_tra, X_val, y_tra, y_val = train_test_split(Xtra, ytra, test_size=0.2)
print(X_tra.shape, y_tra.shape, X_val.shape, y_val.shape)

(1600, 50, 1) (1600, 1) (400, 50, 1) (400, 1)


In [5]:
def train(epochs, batch_size, stateful=True):
    print('Creating Model...')
    model = Sequential()
    model.add(LSTM(10, batch_input_shape=(batch_size, 1, 1), stateful=stateful))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    try:
        tr_batches = X_tra.shape[0] // batch_size
        te_batches = X_val.shape[0] // batch_size
        print('\nTraining for {} epochs: {} batches of {} samples each.'.format(epochs, tr_batches, batch_size))
        print('___________________________________')
        for epoch in range(epochs):
            mean_tr_loss = []
            mean_tr_acc = []
            for j in range(tr_batches):
                j0 = j * batch_size
                j1 = j0 + batch_size
                for i in range(seq_size):
                    x_b = X_tra[j0:j1, i:i+1, 0:1]
                    y_b = y_tra[j0:j1, 0]
                    tr_loss, tr_acc = model.train_on_batch(x_b, y_b)
                    mean_tr_acc.append(tr_acc)
                    mean_tr_loss.append(tr_loss)
                model.reset_states()

            print('{:3d} training accuracy = {}'.format(epoch, np.mean(mean_tr_acc)))
            print('                 loss = {}'.format(np.mean(mean_tr_loss)))
            print()

            mean_te_acc = []
            mean_te_loss = []
            for j in range(te_batches):
                j0 = j * batch_size
                j1 = j0 + batch_size
                for i in range(seq_size):
                    x_b = X_val[j0:j1, i:i+1, 0:1]
                    y_b = y_val[j0:j1, 0]
                    te_loss, te_acc = model.test_on_batch(x_b, y_b)
                    mean_te_acc.append(te_acc)
                    mean_te_loss.append(te_loss)
                model.reset_states()

            print('    validation accuracy = {}'.format(np.mean(mean_te_acc)))
            print('                   loss = {}'.format(np.mean(mean_te_loss)))
            print('___________________________________')

    except KeyboardInterrupt:
        print('\nKeyboard Interrupt!!')

In [6]:
train(5, 100)

Creating Model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (100, 10)                 480       
_________________________________________________________________
dense_1 (Dense)              (100, 1)                  11        
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________

Training for 5 epochs: 16 batches of 100 samples each.
___________________________________
  0 training accuracy = 0.625512421131134
                 loss = 0.6355300545692444

    validation accuracy = 1.0
                   loss = 0.12141431868076324
___________________________________
  1 training accuracy = 1.0
                 loss = 0.038812316954135895

    validation accuracy = 1.0
                   loss = 0.01693037897348404
___________________________________
  2 training accuracy = 1.0
                 l

In [7]:
train(5, 100, False)

Creating Model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (100, 10)                 480       
_________________________________________________________________
dense_2 (Dense)              (100, 1)                  11        
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________

Training for 5 epochs: 16 batches of 100 samples each.
___________________________________
  0 training accuracy = 0.5139874219894409
                 loss = 0.692491888999939

    validation accuracy = 0.5320499539375305
                   loss = 0.6915138363838196
___________________________________
  1 training accuracy = 0.5148999691009521
                 loss = 0.6916003227233887

    validation accuracy = 0.5320499539375305
                   loss = 0.6905477643013
___________________________________
  2 trai